## Open the dataset:

In [4]:
import pandas as pd

df = pd.read_json( 'Arts_crafts.json',lines=True)

df.sample(10)


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
79099,5,True,"09 30, 2016",A3FWTV1T0E2L6X,B0016P086O,NaN,Rivka A. Strom,Use almost everyday.,Five Stars,1475193600,NaN,NaN
306691,5,True,"05 30, 2014",A343RU4APYYS1V,B00E77YAN0,NaN,Rathburn&#039;s Den,These will go well with my Steampunk products....,Antiqued Brass Plated Tokens,1401408000,NaN,NaN
16607,5,True,"12 15, 2014",AD07K1697U80V,B000BGSZCS,{'Color:': ' Series 3'},rox,Great,Five Stars,1418601600,NaN,NaN
433182,5,True,"10 31, 2017",A25SKK3491HWVL,B0020S13HY,NaN,Memories4evr,Nice starter kit to learn pergamano. Fast shi...,Pregnant Parchment Starter Kit,1509408000,NaN,NaN
250141,5,True,"04 20, 2017",A2Z3B4WWFWSJGG,B001B2P4OI,{'Color:': ' Medium Flesh'},Zuchick,NC,Five Stars,1492646400,NaN,NaN
237527,5,True,"03 26, 2015",A1T2DNKLHYWP80,B005P1T20I,{'Size:': ' By The Yard'},Joan Parker,I was very satisfied with the pliers. They ap...,Five Stars,1427328000,NaN,NaN
141011,3,True,"11 27, 2013",ALL6HPZ61K627,B001CE3CGM,{'Color:': ' Original Version'},Yuki,"if you are using those long bigz die, you will...",extra items to buy but for long bigz die is a ...,1385510400,NaN,NaN
382360,1,True,"04 26, 2017",A1X55NSDOI6WA9,B011B8KVZ8,NaN,Michele G.,The foot seems off-center. My needle hits ver...,Doesn't work for me.,1493164800,NaN,NaN
211388,5,True,"06 8, 2015",A23L3LMIK0IT40,B004G5PDSE,"{'Size:': ' DESIGN 1', 'Package Quantity:': ' 1'}",vicki loichinger,I love this product and the Seller was awesome,Five Stars,1433721600,NaN,NaN
70640,5,True,"02 13, 2016",A2O8GLBKIAZ38J,B0013JRFUA,"{'Size:': ' 8-1/2"" x 11""', 'Color:': ' Natural'}",strugglebuggy,I see that these are intended for scrapbooking...,Great for doodling in ink and marker,1455321600,42,[https://images-na.ssl-images-amazon.com/image...


## Get the Bing Liu lexicon

In [5]:
from sklearn import preprocessing

import nltk

nltk.download('opinion_lexicon')

from nltk.corpus import opinion_lexicon

from nltk.tokenize import word_tokenize

print('Total number of words in opinion lexicon', len(opinion_lexicon.words()))

print('Examples of positive words in opinion lexicon',

     opinion_lexicon.positive()[:10])

print('Examples of negative words in opinion lexicon',

     opinion_lexicon.negative()[:10])


Total number of words in opinion lexicon 6789
Examples of positive words in opinion lexicon ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']
Examples of negative words in opinion lexicon ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/peter/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


## Create the “Dictionary”

In [6]:
#Let's create a dictionary which we can use for scoring our review text

nltk.download('punkt')

df.rename(columns={"reviewText": "text"}, inplace=True)

pos_score = 1

neg_score = -1

word_dict = {}


# Adding the positive words to the dictionary

for word in opinion_lexicon.positive():

     word_dict[word] = pos_score


# Adding the negative words to the dictionary

for word in opinion_lexicon.negative():

     word_dict[word] = neg_score


[nltk_data] Downloading package punkt to /Users/peter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Function to do the math

In [19]:
def bing_liu_score(text):

    sentiment_score = 0

    bag_of_words = word_tokenize(text.lower())

    for word in bag_of_words:

         if word in word_dict:

               sentiment_score += word_dict[word]

    return sentiment_score


## Make sure all texts are non-empty and are type text then call the function

In [16]:
df['text'].fillna('no review', inplace=True)

df['Bing_Liu_Score'] = df['text'].apply(bing_liu_score)

## Let’s see the output

In [17]:
df[['overall',"text", 'Bing_Liu_Score']].head(10)

,overall,text,Bing_Liu_Score
0,4,Contains some interesting stitches.,1
1,5,I'm a fairly experienced knitter of the one-co...,22
2,4,Great book but the index is terrible. Had to w...,0
3,5,I purchased the Kindle edition which is incred...,4
4,5,Very well laid out and very easy to read.\n\nT...,5
5,5,"Beginning her career as a freelance knitter, M...",15
6,5,This is a terrific stitch handbook (and I have...,9
7,4,The book needs to be coil bound. The content i...,1
8,5,I really am enjoying this book! I like the siz...,12
9,5,Just received this book and looked over it cov...,6


In [18]:
df.groupby('overall').agg({'Bing_Liu_Score':'mean'})

,Bing_Liu_Score
overall,
1,-0.255049
2,0.566098
3,1.158796
4,2.028146
5,2.130005
